# Image Processing SS 20 - Assignment - 01

### Deadline is 29.4.2020 at 12:00

Please solve the assignments together with a partner. Write the names of yourself and your partner and a link to your repository in a text-submission in the Whiteboard. You have to do this EVERY WEEK.
I will run every notebook. Make sure the code runs without errors, when clicking `Kernel` -> `Restart & Run All`.


## Introduction to Python / Numpy

* [Learn Python in 15 minutes](https://learnxinyminutes.com/docs/python3/): We will use Python 3.
* [Numpy for Matlab Users](https://docs.scipy.org/doc/numpy/user/numpy-for-matlab-users.html#general-purpose-equivalents)
* [Numpy Quickstart](https://docs.scipy.org/doc/numpy/user/quickstart.html)

## Libraries

We will use the following libraries:

* matplotlib
* numpy
* scipy
* skimage


# Exercise 0 - Setup Development Enviroment - [1 Point]

Find a partner, follow the steps in the [README](https://github.com/BildverarbeitungSS20/Hausaufgaben/blob/master/README.md) and paste a link to your repository and your names into the KVV assignment box. Also include your names inside your solution.
You do not need to upload any files to MyCampus Whiteboard. I will clone your repository.

In [ ]:
# display the plots inside the notebook
%matplotlib inline

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pylab
pylab.rcParams['figure.figsize'] = (12, 12)   # This makes the plot bigger

The [skimage](http://scikit-image.org/) library comes with multiple useful test images.  Let's start with an image of an astronaut. 

In [ ]:
from skimage.data import astronaut

In [ ]:
img = astronaut()   # Get the image
print(img.shape)    # the dimension of the image
print(img.dtype)    # the image type

We have a `(512, 512, 3)` array of unsigned bytes. At `img[x, y]` there are three values for R,G and B.

We will always work with floating point arrays between 0 and 1. 

In [ ]:
img = img / 255.

Lets display the image.

In [ ]:
plt.imshow(img)
plt.show()

This is [Eileen Collins](https://en.wikipedia.org/wiki/Eileen_Collins). She was the first astronaut 
 to fly the Space Shuttle through a complete 360-degree pitch maneuver. What an inspiring woman.

## Exercise 1 - Plot -       [1 Point]

Plot the R, G and B channels separately.

In [ ]:
img_red = img.copy()
img_red[:,:,1] = 0
img_red[:,:,2] = 0

img_green = img.copy()
img_green[:,:,0] = 0
img_green[:,:,2] = 0

img_blue = img.copy()
img_blue[:,:,0] = 0
img_blue[:,:,1] = 0

plt.figure(figsize=(18, 12))
plt.subplot(131)
plt.imshow(img_red)
plt.subplot(132)
plt.imshow(img_green)
plt.subplot(133)
plt.imshow(img_blue)
plt.show()


## Exercise 2 - RGB to HSV [6 Points]

Implement the `rgb_to_hsv` and `hsv_to_rgb` functions. Don't use any color conversion functions from a library.


In [ ]:
def rgb_to_hsv(x):
    # maximum color channel
    maxs = np.amax(x, -1)
    # special case for 0,0,0
    non_zero = maxs > 0
    # peak to peak => max - min
    ptp = np.ptp(x, -1)
    # check that ptp is > 0 to avoid divions by zero
    non_zero = ptp > 0

    # max red
    idx_red = (x[..., 0] == maxs) & non_zero
    x[idx_red, 0] = 0.0 + (x[idx_red, 1] - x[idx_red, 2]) / ptp[idx_red]

    # max green
    idx_green = (x[..., 1] == maxs) & non_zero
    x[idx_green, 0] = 2.0 + (x[idx_green, 2] - x[idx_green, 0]) / ptp[idx_green]

    # max blue
    idx_blue = (x[..., 2] == maxs) & non_zero
    x[idx_blue, 0] = 4.0 + (x[idx_blue, 0] - x[idx_blue, 1]) / ptp[idx_blue]

    # s = (MAX-MIN) / MAX
    s = np.zeros_like(ptp)
    s[non_zero] = ptp[non_zero] / maxs[non_zero]

    # scale to [0,1]
    x[..., 0] = (x[..., 0] / 6.0) % 1.0
    x[..., 1] = s
    x[..., 2] = maxs

    return x

In [ ]:
def hsv_to_rgb(x):
    """
    Converts the numpy array `x` from HSV to the RGB. 
    """
    h = x[:,:, 0]
    s = x[:,:, 1]
    v = x[:,:, 2]

    # we assume h to be in [0, 1]
    h_i = (h * 6.0).astype(int)
    f = (h * 6.0) - h_i

    p = v * (1.0 - s)
    q = v * (1.0 - s * f)
    t = v * (1.0 - s * (1.0 - f))

    r = np.empty_like(h)
    g = np.empty_like(h)
    b = np.empty_like(h)

    cond = h_i % 6 == 0
    r[cond] = v[cond]
    g[cond] = t[cond]
    b[cond] = p[cond]

    cond = h_i == 1
    r[cond] = q[cond]
    g[cond] = v[cond]
    b[cond] = p[cond]

    cond = h_i == 2
    r[cond] = p[cond]
    g[cond] = v[cond]
    b[cond] = t[cond]

    cond = h_i == 3
    r[cond] = p[cond]
    g[cond] = q[cond]
    b[cond] = v[cond]

    cond = h_i == 4
    r[cond] = t[cond]
    g[cond] = p[cond]
    b[cond] = v[cond]

    cond = h_i == 5
    r[cond] = v[cond]
    g[cond] = p[cond]
    b[cond] = q[cond]

    # special case s is zero
    cond = s == 0.0
    r[cond] = v[cond]
    g[cond] = v[cond]
    b[cond] = v[cond]

    x[:,:,0] = r
    x[:,:,1] = g
    x[:,:,2] = b
    return x

Plot the saturation of the astronaut image

In [ ]:
img_as_hsv = rgb_to_hsv(img)

plt.imshow(img_as_hsv[:,:,1], cmap='gray')
plt.show()

Increase the saturation by a factor of 2, convert it back to RGB and plot the result.

In [ ]:
img_as_hsv[:,:, 1] = (img_as_hsv[:,:, 1] * 2.0).clip(0.0, 1.0)
img_as_rgb = hsv_to_rgb(img_as_hsv)

plt.imshow(img_as_rgb)
plt.show()

## Exercise 3 - Calculation [2 Points]

In the figure below you can see the [CIE-XYZ](https://de.wikipedia.org/wiki/CIE-Normvalenzsystem) color space.
![](https://upload.wikimedia.org/wikipedia/commons/4/49/CIE-Normfarbtafel.png)

What are the approximate x,y,z values for the following Adobe RGB colors:
* `(0, 0.5, 0.5)`
* `(0.33, 0.33, 0.33)`

A sodium-vapor lamp shines with double the intensity of a mercury-vapor lamp
. The light from the sodium lamp only contains 
the spectral line at `589,00nm` and the light from the mercury lamp only the
spectral line at `435,83 nm`.

What color does a human experience? What are the approximate x,y,z values?

In [ ]:
T = np.array([[2.04159, -0.56501, -0.34473],[-0.96924, 1.87597, 0.04156],[0.01344, -0.11836, 1.01517]])

In [ ]:
from numpy.linalg import inv

ARGB1 = np.array([0, 0.5, 0.5])
ARGB2 = np.array([0.33, 0.33, 0.33])

XYZ1 = inv(T).dot(ARGB1)
print(XYZ1)

XYZ2 = inv(T).dot(ARGB2)
print(XYZ2)

For a sodium-vapor lamp and a mercury-vapor lamp a human does experience a bright red.
The approximate x,y,z values are 0.5, 0.32, 0.18
You can read the values directly from the XYZ-diagram. Connect the two spectral lines with a line.
The resulting color is on this line.
As the two colors mix and the sodium lamp has double intensity, the color that a human experience is nearer to
the sodium lamp.
It is exactly 1/3 of the total line length from the sodium spectral line away.